In [1]:
import time 

class TimerError(Exception):
  """A custom exception used to report errors in use of Timer class"""

class Timer:
  def __init__(self):
    self._start_time= None
    self._elapsed_time= None
  
  def start(self):
    """Start a new timer"""
    if self._start_time is not None:
      raise TimerError("Timer is running. Use .stop()")
    self._start_time= time.perf_counter()
  
  def stop(self):
    """Save the elapsed time and re-initialize timer"""
    if self._start_time is None:
      raise TimerError("Timer is not running. Use start()")
    self._elapsed_time= time.perf_counter() - self._start_time
    self._start_time= None

  def elapsed(self):
    """Report elapsed time"""
    if self._elapsed_time is None:
      raise TimerError("Timer has not been run yet. Use start()")
    return self._elapsed_time

  def __str__(self):
    """print() prints elapsed time"""
    return str(self._elapsed_time)

## **Quick Sort**
**T(n) is O(n log n)**

Uses divide and conquer <br>
Sorts in place

Worst case complexity(sorted array) is O(n^2) <br>
Randomly choosing the pivot is a good strategy to beat worst case inputs

Partitioning wrt the pivot takes time O(n) <br>
If the pivot is the median <br>
*T(n) = 2T(n/2)+n* <br>
*T(n)* is *O(n log n)*

In [2]:
import sys
sys.setrecursionlimit(2**31-1)

In [3]:
def quick_sort(L,l,r):
  if (r-1<=1):
    return
  
  pivot,lower,upper= L[l],l+1,l+1
  for i in range(l+1,r):
    if L[i]>pivot:   # Extend upper segment
      upper+=1
    else:            # Exchange L[i] with start of upper segment
      L[i],L[lower]= L[lower],L[i]
      # Shift both segments
      lower+=1
      upper+=1
    # Move pivot between lower and upper
    L[l],L[lower-1]= L[lower-1],L[l]
    lower-=1
    # Recursive calls
    quick_sort(L,l,lower)
    quick_sort(L,lower+1,upper)
    return L

### Performance

In [4]:
import random
inputlists={}
inputlists['random']= [random.randrange(100000000) for i in range(1000000)]
inputlists['ascending']= [i for i in range(1000000)]
inputlists['descending']= [i for i in range(999999,-1,-1)]

t=Timer()
for k in inputlists.keys():
  tlist= inputlists[k][:]
  t.start()
  quick_sort(tlist, 0, len(tlist))
  t.stop()
  print(k,t)

random 1.518199951533461e-05
ascending 1.4246000318962615e-05
descending 1.4051999642106239e-05


# **Difference between Lists and Arrays**

## **Lists**

* Flexible length
* Easy to modify the structure
* Values are scattered in memory

* Typically a sequence of nodes
* Each node contains a value and points to the next node in the sequence
* Easy to modify
   - Inserting and deleting is easy
   - Flexible size
* Need to follow links to access *A[ i ]*
   - Takes time O(i)

## **Arrays**

* Fixed size
* Allocate a contiguous block of memory
* Supports random access



* Fixed size, declared in advance
* Allocate a contiguous block of memory
   - n times the storage for a single value
* "Random" access
   - Compute offset to *A[ i ]* from *A[ 0 ]*
   - Accessing *A[ i ]* takes constant time, independent of i
* Inserting and deleting elements is expensive
   - Expanding and contracting requires moving O(n) elements in the worst case

## **Operations**

* Exchange *A[ i ]* and *A[ j ]*
  - Constant time for arrays
  - O(n) for lists
* Delete *A[ i ]*, insert *v* after *A[ i ]*
  - Constant time for lists if we are already at *A[ i ]*
  - O(n) for arrays
* Need to keep implementation in mind when analyzing data structures
  - For instance, can we use binary search to insert in a sorted sequence?
  - Either search is slow, or the insertion is slow, still O(n)


## **Summery**

* Sequences can be stored as lists or arrays
* Lists are flexible but accessing element is O(n)
* Arrays suport random access but are difficult to expand, contract
* Algorithm analysis needs to take into account the underlying implementation

## **Designing a Flexible List**

* Python class *Node*
* A list is a sequence of nodes
  - *self.value* is the stored value
  - *self.next* points to next node
* Empty list?
  - *self.value* is *None*
* Creating lists
  - *l1= Node()*  ---- empty list
  - *l2= Node(5)*  ---- singleton list
  - *l1.isempty() --- True*
  - l2.isempty() --- False
* Appending to a list (Add *v* to the end of list *l*)
  - Normal implementation
    - If *l* is empty, update *l.value* from *None* to *v*
    - If at last value (*l.next* is *None*), point *next* at new node with value *v*
    - Otherwise recursively append to rest of list
  - Iterative implementation
     - If empty, replace *l.value* by *v*
     - Loop through *l.next* to end of list
     - Add v at the end of the list
  - Inserting an element
    - Create a new node with *v*
    - Exchange the values *v0*, *v*
    - Make new node point to *head.next*
    - Make *head.next* point to new node

* Deleting from a list (Remove first occurance of *v*)
  - Scan list for first v --- look ahead at next node
  - If next node value is v, bypass it
  - Can't bypass the first node in the list
    - Instead copy the second node value to head
    - Bypass second node


In [5]:
class Node:
  def __init__(self, v=None):     # Initializing empty list
    self.value= v
    self.next= None
  
  def isempty(self):
    if self.value== None:
      return True
    else:
      return False
  
  def append(self,v):
    if self.isempty():
      self.value=v
    elif self.next== None:
      self.next= Node(v)
    else:
      self.next.append(v)
    return

  def append(self,v):
    if self.isempty():
      self.value=v
      return

    temp=self
    while temp.next !=None:
      temp=temp.next
    
    temp.next= Node(v)
    return

  def insert(self,v):
    if self.isempty():
      self.value=v
      return
    
    newnode= Node(v)

    # Exchange values in self and newnode
    self.value, newnode.value = newnode.value, self.value

    # Switch links
    self.next, newnode.next = newnode, self.next
    return

  def delete(self,v):
    if self.isempty():
      return
    
    if self.value==v:
      self.value= None
      if self.next != None:
        self.value= self.next.value
        self.next= self.next.next
      return
    
    else:
      if self.next != None:
        self.next.delete(v)
        if self.next.value== None:
          self.next= None
    return

# **Lists in Python**

* Python lists are not implemented as flexible linked lists
* Underlying interpretation maps the list to an array
   - Assign a fixed block when you create a list
   - Double the size if the list overflows the array
* Keep track of the last position of the list in the array
   - *l.append()* and *l.pop()* are constant time, amortised -- *O(1)*
   - Insertion/deletion require time *O(n)*
* Effectively, Python lists behave more like arrays than lists

# **Arrays vs Lists in Python**

* Arrays are useful for representing matrices
* In list notation, these are nested lists
* Need to be careful when initializing a multidimentional list

In [6]:
zerolist=[0,0,0]
zeromatrix= [zerolist,zerolist,zerolist]

zeromatrix[1][1]=1
print(zeromatrix)

# Mutability aliases different values

[[0, 1, 0], [0, 1, 0], [0, 1, 0]]


In [7]:
zeromatrix= [ [0 for i in range(3)] for j in range(3)]

zeromatrix[1][1]=1
print(zeromatrix)

[[0, 0, 0], [0, 1, 0], [0, 0, 0]]


# **Numpy Arrays**

* The Numpy library provides arrays as a basic type.




In [8]:
import numpy as np
zeromatrix= np.zeros(shape=(3,3))
print(zeromatrix)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


* Can create array from any sequence type

In [9]:
newarray= np.array([[0,1],[1,0]])
print(newarray)

[[0 1]
 [1 0]]


* *arange* is the equivalent of *range* for lists

In [10]:
row2= np.arange(5)
row2

array([0, 1, 2, 3, 4])

* Can operate on a matrix as a whole

In [11]:
A= np.array([[1,0,0],[0,2,3],[1,4,7]])
B= np.array([[2,1,1],[1,1,0],[0,1,0]])

In [12]:
C=3*A+B
C

array([[ 5,  1,  1],
       [ 1,  7,  9],
       [ 3, 13, 21]])

In [13]:
C= np.matmul(A,B)
C

array([[ 2,  1,  1],
       [ 2,  5,  0],
       [ 6, 12,  1]])

# **Dictionary**

* An array/list allows access through positional indices
* A dictionay allows access through arbitary keys
  - A collection of key-value pairs
  - Random access -- access time is the same for all keys

## **Implementing a dictionary**

* The underlying storage is an array
  - Given an offset *i*, find *A[i]* in constant time
* Keys have to be mapped to {0,1,...,n-1}
  - Given an key *k*, convert it to an offset *i*

* **Hash Function**
  - *h : S->X* maps a set of values *S* to a small range of integers *X= {0,1,...,n-1}*
  - Typically |X| << |S| , so there will be collisions, *h(s)= h(s'), s!=s'*
  - A good hash function will minimize collisions
  - Hash function maps *s* and *s'* to the same value even though *s* is not equal to *s'*
  - SHA-256 is an industry standard hashing function whose range is 256 bits
    - Use to hash large files -- avoid uploading duplicates to cloud storage

# **Hash Table**

- An array *A* of size *n* combined with a hash function which maps keys to the range from *0* to *n-1*
- *h* maps keys to {0,1,...,n-1}
- Ideally, when we create an entry for key *k*, *A[h(k)]* will be unused
  - But what if there's already a value at that location?
- Dealing with collisions
  - Open addressing (closed hashing)
     - Probe a sequence of alternate slots in the same array
  - Open hashing
     - Each slot in the array points to a list of values
     - Insert into the list for the given slot